### 深度神经网络

前面提到的都是浅层神经网络。接下来更深入一些学习深度神经网络。

使用DNN会遇到一些问题：

- 梯度弥散 vs. 梯度爆炸
- 训练很慢
- 拥有几百万参数的模型很容易过拟合训练集

问题导向，并提出相应的解决方法，都体现在代码中。绝不空谈。

终于来到了深度学习的世界！

本章主要内容有：

- 梯度问题
    - Xavier and He初始化
    - 不饱和激活函数
    - BN
    - 梯度剪切
- 重用预训练层
    - 重用TF模型
    - 重用其他框架模型
    - 冻结低层layer
    - 缓存冻结层
    - Tweaking, Dropping, Replacing高层layer
    - Model动物园
    - 非监督预训练
    - 在辅助任务上预训练
    
- 更快的Optimizer
    - Momentum优化
    - Nesterov加速梯度
    - AdaGrad
    - RMSProp
    - Adam优化
    - 学习率动态调整
- 避免过拟合：正则化
    - 早停止
    - L1,L2正则化
    - Dropout
    - Max-Norm正则化
    - 数据增强
- 实际指导原则
- 练习

### 梯度弥散/爆炸问题

反向传播算法的工作原理是从输出层到输入层，传播误差的梯度值。一旦算法计算出成本函数对每层参数的梯度，算法接着就会使用这些梯度来更新参数。

但是梯度通常会慢慢变小，我们称输出层为high layer,越往输入层，越low.导致的结果就是低层的隐层参数不更新，也就不能收敛到好的结果上来。这就是梯度弥撒问题。

与此相反，梯度爆炸就是指，梯度越来越大，这在RNN中更为常见一些。

总之，DL面临着梯度不稳定的问题。不同的层有不同的学习速度，也即参数更新的速率。

In [1]:
# Setup
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

### Xavier and He初始化

使用tf.layers.dense(), 这个应该就是早期tf.contrib.layers.fully_connected()这个函数。一个稳定一个不稳定。

这个是在矩阵的初始化时选择使用的技巧。真正在TF中用时，直接调用即可。

In [2]:
import tensorflow as tf
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')

In [3]:
he_init = tf.contrib.layers.variance_scaling_initializer()
hidden1 = tf.layers.dense(X,n_hidden1, activation=tf.nn.relu, kernel_initializer=he_init, name='hidden1')

### 非饱和激活函数

- Leaky ReLU
- ELU
- SELU


ReLU并不是完美的，比如它面临着一个问题就是：**dying**.就是说训练过程中，有些神经元die后，只输出0。尤其是在学习率较大的情况下更为严重。一旦开始输出0后，就再也回不来了。这是ReLU激活函数的特征。

但是实际上，这种情况极为少见。

到目前为止，本章的内容我都在论文上有学习过，所以不再继续看本章。我准备先看13章，CNN部分，然后RNN，强化学习，最后看第12章节的分布式TF.